## Import DETR of Facebook

In [ ]:
# Download detr repos
!git clone https://github.com/facebookresearch/detr.git 

In [ ]:
!pip install utils
!pip install glob2
!pip install --upgrade pip

In [ ]:
# basic
import os
import numpy as np 
import pandas as pd 
from tqdm.autonotebook import tqdm

#computer vison module
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

#pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

#for K-fold cross validation
from sklearn.model_selection import StratifiedKFold

#Hungarian loss & bipartite matcher
import sys
sys.path.append('./detr/')

from detr.models.matcher import HungarianMatcher
from detr.models.detr import SetCriterion

#Data aug
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

#Glob
from glob2 import glob

# Data pre-processing

In [ ]:
train_dir = '../input/global-wheat-detection/train/'
test_dir = '../input/global-wheat-detection/test/'

train_annos = '../input/global-wheat-detection/train.csv'

# glob to get lists of files of each dir 
train_fns = glob(train_dir + '*')
test_fns = glob(test_dir + '*')

In [ ]:
print('Number of train images: ', len(train_fns))
print('Number of test images: ', len(test_fns))

In [ ]:
train = pd.read_csv(train_annos)
train

In [ ]:
#Create dataframe with all training images
all_train_images = pd.DataFrame([fns.split('/')[-1][:-4] for fns in train_fns])
all_train_images.columns=['image_id']

#Merge with bboxes
all_train_images = all_train_images.merge(train, on='image_id', how='left')

#Fill nan values with zero
all_train_images['bbox'] = all_train_images.bbox.fillna('[0,0,0,0]')

#Split 4 values of bbox to columns
bbox_items = all_train_images.bbox.str.split(',', expand = True)

all_train_images['x_min'] = bbox_items[0].str.strip('[ ').astype(float)
all_train_images['y_min'] = bbox_items[1].str.strip(' ').astype(float)
all_train_images['width'] = bbox_items[2].str.strip(' ').astype(float)
all_train_images['height'] = bbox_items[3].str.strip('] ').astype(float)

In [ ]:
# Check imgs without bboxes.
print('There are {} images with no bounding box!'.format(len(all_train_images)- len(train)))

In [ ]:
# Remove img without bboxes.
all_train_images = all_train_images[all_train_images.width != 0]

print('There are {} images with no bounding box!'.format(len(all_train_images)- len(train)))

In [ ]:
# This code is used for connecting boxes with id, we partly get from UIT LOGO baseline. 
def get_all_bboxes(df, image_id):
    image_bboxes = df[df.image_id == image_id]   
    bboxes = []
    for _,row in image_bboxes.iterrows():
        bboxes.append((row.x_min, row.y_min, row.width, row.height))
        
    return bboxes

def plot_image_examples(df, rows=3, cols=3):
    fig, axs = plt.subplots(rows, cols, figsize=(25,20))
    for row in range(rows):
        for col in range(cols):
            idx = np.random.randint(len(df), size=1)[0]
            img_id = df.iloc[idx].image_id
            
            img = Image.open(train_dir + img_id + '.jpg')
            axs[row, col].imshow(img)
            
            bboxes = get_all_bboxes(df, img_id)
            
            for bbox in bboxes:
                rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')
                axs[row, col].add_patch(rect)
            
            axs[row, col].axis('off')

In [ ]:
# Sample images with bounding box.
plot_image_examples(all_train_images)

In [ ]:
# Drop old bbox column
all_train_images.drop(columns = ['bbox'], inplace=True)

# Count bbox with condition
all_train_images['count'] = all_train_images.apply(lambda row: 1 if np.isfinite(row.width) else 0, axis=1)

all_train_images

## Creating Fold

In [ ]:
# Config
n_folds = 4
seed = 42
num_classes = 2 #(wheat and no-obj class)
num_queries = 100
null_class_coef = 0.5
BATCH_SIZE = 12
LR = 2e-5
EPOCHS = 20

In [ ]:
"""[1]"""

skf = StratifiedKFold(n_splits= n_folds, shuffle=True, random_state=42)

df_folds = all_train_images[['image_id', 'count']].copy()

#Gather all boxes of each img 
df_folds = df_folds.groupby('image_id').count()

#Get source for each img, we stratify to 4 folds.
df_folds.loc[:, 'source'] = all_train_images[['image_id', 'source']].groupby('image_id').min()['source']
df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['count'].apply(lambda x: f'_{x // 15}').values.astype(str)
    )

df_folds.loc[:, 'fold'] = 0

for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

In [ ]:
df_folds

## Create dataset object

In [ ]:
"""[2]"""
class WheatDataset(Dataset):
    
    def __init__(self,image_ids,dataframe,transforms=None):
        self.image_ids = image_ids
        self.df = dataframe        
        
        #Data augmentation 
        self.transforms = transforms
        
        
    def __len__(self) -> int:
        return self.image_ids.shape[0]
    
    def __getitem__(self,index):
        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]
        
        image = cv2.imread(f'{train_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        # DETR takes in data in coco format 
        boxes = records[['x_min', 'y_min', 'width', 'height']].values
        
        #Area of bb
        area = boxes[:,2]*boxes[:,3]
        area = torch.as_tensor(area, dtype=torch.float32)
        
        labels =  np.zeros(len(boxes), dtype=np.int32)
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            boxes = sample['bboxes']
            labels = sample['labels']            
            
        #Normalizing BBOXES            
        _,h,w = image.shape
        boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels,dtype=torch.long)
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        
        return image, target, image_id

## Create DETR object

In [ ]:
"""[3]"""
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        #Download pretrained DETR model with backbone resnet101 feature extractor.
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet101', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

In [ ]:
# Pretrained DETR with frozen transformer.
class frozen_transformer_DETR(nn.Module):
    def __init__(self,num_classes, num_queries):
        super(frozen_transformer_DETR, self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        #Download pretrained DETR model with backbone resnet101 feature extractor.
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet101', pretrained=True)
        
        
        self.model.transformer.encoder.layers.requires_grad = False
        self.model.transformer.decoder.layers.requires_grad = False
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes)
        self.model.num_queries = self.num_queries
        
        
    def forward(self,images):
        return self.model(images)

In [ ]:
#Create matcher for loss calculation and set "lambda" params
# loss_ce: classification loss (Hungarian matching loss)
# loss_bbox: bbox loss
# giou: giou loss

matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}

losses = ['labels', 'boxes', 'cardinality']

## Data Augmentation

In [ ]:
"""[4]"""
def get_train_transforms():
    return A.Compose([A.OneOf([A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, val_shift_limit=0.2, p=0.9),                               
                      A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.9)],p=0.9),                      
                      A.ToGray(p=0.01),                      
                      A.HorizontalFlip(p=0.5),                      
                      A.VerticalFlip(p=0.5),                      
                      A.Resize(height=512, width=512, p=1),                      
                      A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),                      
                      ToTensorV2(p=1.0)],                      
                      p=1.0,                     
                      bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
                      )


def get_valid_transforms():
    return A.Compose([A.Resize(height=512, width=512, p=1.0),
                      ToTensorV2(p=1.0)], 
                      p=1.0, 
                      bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
                      )

## Loss calculation object

In [ ]:
# To compute and store avg and current value.
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_fn(data_loader,model,criterion,optimizer,device,scheduler,epoch):
    
    model.train()
    criterion.train()
    
    summary_loss = AverageMeter()
    
    with tqdm(data_loader, total=len(data_loader)) as tk0:
    
        for step, (images, targets, image_ids) in enumerate(tk0):
        
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        

            output = model(images)
        
            loss_dict = criterion(output, targets)
            weight_dict = criterion.weight_dict
        
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        
            optimizer.zero_grad()

            losses.backward()
            optimizer.step()
            
            if scheduler is not None:
                scheduler.step()
        
            summary_loss.update(losses.item(),BATCH_SIZE)
            tk0.set_postfix(loss=summary_loss.avg)
        
    return summary_loss

In [ ]:
def eval_fn(data_loader, model,criterion, device):
    
    model.eval()
    criterion.eval()
    
    summary_loss = AverageMeter()
    
    # Disabled gradient calculation over the weights
    with torch.no_grad():
        
        tk0 = tqdm(data_loader, total=len(data_loader))
        for step, (images, targets, image_ids) in enumerate(tk0):
            
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            output = model(images)
        
            loss_dict = criterion(output, targets)
            weight_dict = criterion.weight_dict
        
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
            
            summary_loss.update(losses.item(),BATCH_SIZE)
            tk0.set_postfix(loss=summary_loss.avg)
    
    return summary_loss

# Training

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
"""[5]"""
def get_data_loader(fold):    
    df_train = df_folds[df_folds['fold'] != fold]
    df_valid = df_folds[df_folds['fold'] == fold]

    train_dataset = WheatDataset(
        image_ids=df_train.index.values,
        dataframe=all_train_images,
        transforms=get_train_transforms()
        )

    train_data_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
        )

    valid_dataset = WheatDataset(
        image_ids=df_valid.index.values,
        dataframe=all_train_images,
        transforms=get_valid_transforms()
        )

    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
        )
    
    return train_data_loader, valid_data_loader

In [ ]:
def trainnnn(train_data_loader, valid_data_loader, device, model, criterion, optimizer, fold, best_loss, index):
    for epoch in range(EPOCHS):
        train_loss = train_fn(train_data_loader, model,criterion, optimizer,device,scheduler=None,epoch=epoch)
        valid_loss = eval_fn(valid_data_loader, model,criterion, device)
        
        print('|EPOCH {}| TRAIN_LOSS {}| VALID_LOSS {}|'.format(epoch+1,train_loss.avg,valid_loss.avg))
        
        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            print('Best model found for Fold {} in Epoch {}........Saving Model'.format(fold,epoch+1))
            torch.save(model.state_dict(), f'THE_BEST_{index}_fold_{fold}.pth')
        
        index+=1

In [ ]:
"""For training the whole Pretrained DETR"""
#Set device, load model and set criteron
device = torch.device('cuda')

model = DETRModel(num_classes=num_classes, num_queries=num_queries)
model = model.to(device)

criterion = SetCriterion(num_classes-1, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
criterion = criterion.to(device)

In [ ]:
"""[OR] For training Pretrained DETR with frozen transformer"""
"""
#Set device, load model and set criteron
device = torch.device('cuda')

model = frozen_transformer_DETR(num_classes=num_classes, num_queries=num_queries)
print('Total parameters of DETR: ', sum(p.numel() for p in model.parameters()))
print('We now fine-tune Transformer by freezing the pretrained model!')
print('Total of params of MLP layers for clases and boxes.', sum(p.numel() for p in model.parameters() if p.requires_grad))
model = model.to(device)

criterion = SetCriterion(num_classes-1, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
criterion = criterion.to(device)
"""

In [ ]:
best_loss = 10**5
index = 0
    
#AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

#Train 4 folds, each fold 10 epochs.
for fold in range(n_folds):
    train_data_loader, valid_data_loader = get_data_loader(fold=fold)
    trainnnn(train_data_loader, valid_data_loader, device, model, criterion, optimizer, fold, best_loss, index)

# Recall and test model 

In [ ]:
#Recall model
device = torch.device("cuda")

model = frozen_transformer_DETR(num_classes = num_classes, num_queries=num_queries)
# model.load_state_dict(torch.load('./THE_BEST_9_fold_3.pth'))
model.load_state_dict(torch.load('../input/v14unfreezen/THE_BEST_0_fold_3.pth'))
model = model.to(device)

criterion = SetCriterion(num_classes-1, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
criterion = criterion.to(device)

## Create test dataset object
To parse annotation of test images

In [ ]:
class WheatTestDataset(Dataset):
    
    def __init__(self, image_dir, dataframe, transforms=None):
        super().__init__()
        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        boxes = records[['xmin', 'ymin', 'width', 'height']].values

        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id, boxes

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
def get_test_transforms():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ])

## Test annotation processing

In [ ]:
test_annos = pd.read_csv('../input/d/zhangshh/test-annotations/_annotations.csv')
test_annos

In [ ]:
seq = '_jpg.rf.'
test_annos['filename'] = [name.split(seq, 1)[0] for name in test_annos['filename']]

test_annos = test_annos.rename(columns = {'filename': 'image_id'})

test_annos.drop(columns = ['class'], inplace=True)
test_annos.drop(columns = ['width'], inplace=True)
test_annos.drop(columns = ['height'], inplace=True)

test_annos['width'] = test_annos['xmax'] - test_annos['xmin']
test_annos['height'] = test_annos['ymax'] - test_annos['ymin']

test_annos.drop(columns = ['xmax'], inplace=True)
test_annos.drop(columns = ['ymax'], inplace=True)

test_annos

In [ ]:
# Get test dataframe 
test_df = test_annos

test_dataset = WheatTestDataset(image_dir = test_dir,
                                dataframe = test_df,  
                                transforms = get_test_transforms())

test_data_loader = DataLoader(test_dataset,
                              batch_size=10,
                              shuffle=False,
                              num_workers=4,
                              drop_last=False,
                              collate_fn=collate_fn
                             )

In [ ]:
"""[6]"""
confidence_threshold = 0.6

def predict_and_view(test_data_loader, model, confidence_threshold): 
    
    images, image_ids, all_test_boxes = next(iter(test_data_loader))
    
    _, h, w = images[0].shape # for DE norm boxes
    
    images = list(image.to(device) for image in images)
    
    all_pred_boxes = []
    
    # Predict 
    with torch.no_grad():
        outputs = model(images)   
    
    outputs = [{k: v.to(device) for k, v in outputs.items()}]    
        
    for i, image in enumerate(images):
        
        # For plotting test annotation boxes. 
        test_boxes = all_test_boxes[i]
        
        sample = image.permute(1,2,0).cpu().numpy()
        
        fig, ax = plt.subplots(1, 1, figsize=(16, 8))
        
        for box in test_boxes:
            color = (220, 0, 0)
            cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2]+box[0], box[3]+box[1]),
                  color, 1)
            # print(box[0], box[1], box[2],box[3])
        
        # For plotting predicted boxes.
        pred_boxes = outputs[0]['pred_boxes'][i].detach().cpu().numpy()
        pred_boxes = [np.array(box).astype(np.int32) for box in A.augmentations.bbox_utils.denormalize_bboxes(pred_boxes, h, w)] # DE norm
        prob  = outputs[0]['pred_logits'][i].softmax(1).detach().cpu().numpy()[:,0]
    
        
        list_pred_boxes = np.empty(shape=[4, 1])
        
        for box, p in zip(pred_boxes, prob):        
            if p > confidence_threshold:
                color = (0,0,220) 
                cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2]+box[0], box[3]+box[1]),
                      color, 1)
                
                # For eval 
                box[2] = box[2]+box[0]
                box[3] = box[3]+box[1]
                
                box = box.reshape(4,1)
                list_pred_boxes = np.concatenate((list_pred_boxes,box),axis=1)
                
        all_pred_boxes.append(list_pred_boxes)
        
        ax.set_axis_off()
        ax.imshow(sample)
        
    return all_pred_boxes, all_test_boxes

In [ ]:
np_all_pred_boxes, np_all_test_boxes = predict_and_view(test_data_loader, model, confidence_threshold=confidence_threshold)

## Evaluation

In [ ]:
all_pred_boxes = []
all_test_boxes = []

for pred_boxes in np_all_pred_boxes:
    # Transpose the matrix of boxes
    pred_boxes = pred_boxes.transpose()
    # Convert to torch tensor
    pred_boxes = torch.from_numpy(pred_boxes)
    all_pred_boxes.append(pred_boxes)
    
for img_test_boxes in np_all_test_boxes:
    
    for box in img_test_boxes:
        box[2] = box[2]+box[0]
        box[3] = box[3]+box[1]
        
    # Convert to torch tensor
    img_test_boxes = torch.from_numpy(img_test_boxes)
    all_test_boxes.append(img_test_boxes)  

In [ ]:
for i in range(10):
    print(all_pred_boxes[i].shape, ' -- ', all_test_boxes[i].shape)

In [ ]:
def find_intersection(box, test_boxes):
    """Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.
    """
    # PyTorch auto-broadcasts singleton dimensions
    lower_bounds = torch.max(box[:, :2].unsqueeze(1), test_boxes[:, :2].unsqueeze(0))  
    upper_bounds = torch.min(box[:, 2:].unsqueeze(1), test_boxes[:, 2:].unsqueeze(0))  
    intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)  
    
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  


def find_iou(box, test_boxes):
    """Find IoU overlap of every box combination between two sets of boxes that are in boundary coordinates.
    """        
    # Find intersections
    intersection = find_intersection(box, test_boxes)  

    # Find areas of each box in both sets
    areas_box = (box[:, 2] - box[:, 0]) * (box[:, 3] - box[:, 1])  
    areas_test_boxes = (test_boxes[:, 2] - test_boxes[:, 0]) * (test_boxes[:, 3] - test_boxes[:, 1]) 

    # Find the union
    union = areas_box.unsqueeze(1) + areas_test_boxes.unsqueeze(0) - intersection  

    return intersection / union 

In [ ]:
def AP(test_boxes, pred_boxes):
    """Calculate AP score of the prediction.
    """
    average_precisions = torch.zeros((0), dtype=torch.float)
    # All considered boxes
    consi_boxes = 0
    if test_boxes.size(0) > pred_boxes.size(0):
        consi_boxes = test_boxes.size(0)
    else:
        consi_boxes = pred_boxes.size(0)
        
    true_pos = torch.zeros((consi_boxes), dtype=torch.float).to(device) 
    false_pos = torch.zeros((consi_boxes), dtype=torch.float).to(device)  
    

    for box in test_boxes:
        this_box = box.unsqueeze(0)
        overlaps = find_iou(this_box, pred_boxes)
        max_overlap, idx = torch.max(overlaps.squeeze(0), dim=0)
        
        # Overlap < 0.3 is not acceptable 
        if max_overlap.item() < 0.3:
            continue
        
        # Need it to pop detected box
        og_id = torch.LongTensor(range(pred_boxes.size(0)))[idx]
        pred_boxes = torch.cat([pred_boxes[0:og_id], pred_boxes[og_id+1:]])
        
        if max_overlap.item() >= 0.5:
            true_pos[idx] = 1
            
        else:
            false_pos[idx] = 1
        
        if pred_boxes.size(0) == 0:
            break
        
    
    sum_true_pos = torch.cumsum(true_pos, dim=0)
    sum_false_pos = torch.cumsum(false_pos, dim=0)
    
    
    # High precision means the accuracy of the bboxes found is high. 
    arr_precision = sum_true_pos / (
                sum_true_pos + sum_false_pos + 1e-10)
    # High recall means the rate of omitting the ground truth positive bboxes is low.
    arr_recall = sum_true_pos/consi_boxes
    
    
    # Order Precison and Recall like the ROC curve
    # Find the mean of the maximum of the precisions corresponding to recalls above the threshold 't'
    recall_thresholds = torch.arange(start=0, end=1.1, step=.1).tolist() # 0.0 ---> 1.0
    precisions = torch.zeros((len(recall_thresholds)), dtype=torch.float).to(device)
    
    for i, t in enumerate(recall_thresholds):
        recalls_above_t = arr_recall >= t
        if recalls_above_t.any():
            precisions[i] = arr_precision[recalls_above_t].max()
        else:
            precisions[i] = 0.0
            
        average_precision = precisions.mean().item()
            
        
        
    return arr_precision, arr_recall, average_precision

In [ ]:
# Display results and find mAP
APs = 0
mAP = 0


for i in range(10):    
    # precision = torch.zeros((), dtype=torch.float)
    # recall = torch.zeros((24), dtype=torch.float)
    
    precision, recall, average_precision = AP(all_pred_boxes[i], all_test_boxes[i])
    num_predict, num_test = all_pred_boxes[i].size(0), all_test_boxes[i].size(0)
    
    APs += average_precision
    mAP = APs/10
    
    print('Image {}:'.format(i+1), '\n', 'Number of Predicted boxes:', num_predict, end='\n')
    print(' Number of Ground truth boxes:', num_test, end='\n')
    
    # print('Precision score: ',precision, end = '\n')       
    # print('Rcall score: ', recall, end = '\n')
    
    
    print('AP score: ',average_precision, end = '\n')
    print('------------------------------------', end ='\n')
    
print('mAP score: ',mAP, end = '\n')

In [ ]:
torch.cuda.empty_cache()

# References

[1] https://www.kaggle.com/shonenkov/training-efficientdet

[2],
[3],
[4],
[5]
https://www.kaggle.com/tanulsingh077/end-to-end-object-detection-with-transformers-detr

[6] https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-train
